## Direct API calls 

In [ ]:
import os
import requests
import json
from dotenv import load_dotenv

# https://docs.perplexity.ai/api-reference/chat-completions

In [ ]:
# Load environment variables from .env file
load_dotenv()
perplexity_api_key = os.getenv("PERPLEXITY_API_KEY")

### Perplexity

In [8]:
url = "https://api.perplexity.ai/chat/completions"

payload = {
    "model": "sonar",
    "messages": [
        {
            "role": "system",
            "content": "Be precise and concise."
        },
        {
            "role": "user",
            "content": "How many stars are there in our galaxy?"
        }
    ]
}
headers = {
    "Authorization": f"Bearer {perplexity_api_key}",
    "Content-Type": "application/json"
}

In [9]:

response = requests.request("POST", url, json=payload, headers=headers)

In [16]:
response.text

'{"id": "e7e777c1-db5e-4ce3-b258-f44a5f5d9605", "model": "sonar", "created": 1747745169, "usage": {"prompt_tokens": 14, "completion_tokens": 40, "total_tokens": 54, "search_context_size": "low"}, "citations": ["https://en.wikipedia.org/wiki/Milky_Way", "https://www.esa.int/Science_Exploration/Space_Science/Herschel/How_many_stars_are_there_in_the_Universe", "https://www.space.com/25959-how-many-stars-are-in-the-milky-way.html", "https://www.youtube.com/watch?v=Py2nZYmvTKg", "https://science.nasa.gov/universe/stars/"], "object": "chat.completion", "choices": [{"index": 0, "finish_reason": "stop", "message": {"role": "assistant", "content": "The Milky Way galaxy contains an estimated 100 to 400 billion stars. This range accounts for the difficulty in detecting all stars, especially faint or low-mass ones[1][3][4]."}, "delta": {"role": "assistant", "content": ""}}]}'

In [ ]:
# Check if response.text is a string
print(f"Type of response.text: {type(response.text)}")

# Parse the JSON string into a Python dictionary
response_dict = json.loads(response.text)

# Print the formatted JSON with indentation for better readability
print(json.dumps(response_dict, indent=2))

# You can now access the dictionary elements directly
if 'choices' in response_dict and len(response_dict['choices']) > 0:
    message = response_dict['choices'][0]['message']
    print(f"\nResponse content: {message['content']}")

Type of response.text: <class 'str'>
{
  "id": "e7e777c1-db5e-4ce3-b258-f44a5f5d9605",
  "model": "sonar",
  "created": 1747745169,
  "usage": {
    "prompt_tokens": 14,
    "completion_tokens": 40,
    "total_tokens": 54,
    "search_context_size": "low"
  },
  "citations": [
    "https://en.wikipedia.org/wiki/Milky_Way",
    "https://www.esa.int/Science_Exploration/Space_Science/Herschel/How_many_stars_are_there_in_the_Universe",
    "https://www.space.com/25959-how-many-stars-are-in-the-milky-way.html",
    "https://www.youtube.com/watch?v=Py2nZYmvTKg",
    "https://science.nasa.gov/universe/stars/"
  ],
  "object": "chat.completion",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "The Milky Way galaxy contains an estimated 100 to 400 billion stars. This range accounts for the difficulty in detecting all stars, especially faint or low-mass ones[1][3][4]."
      },
      "delta": {
        "role

In [20]:
response.json()

{'id': 'e7e777c1-db5e-4ce3-b258-f44a5f5d9605',
 'model': 'sonar',
 'created': 1747745169,
 'usage': {'prompt_tokens': 14,
  'completion_tokens': 40,
  'total_tokens': 54,
  'search_context_size': 'low'},
 'citations': ['https://en.wikipedia.org/wiki/Milky_Way',
  'https://www.esa.int/Science_Exploration/Space_Science/Herschel/How_many_stars_are_there_in_the_Universe',
  'https://www.space.com/25959-how-many-stars-are-in-the-milky-way.html',
  'https://www.youtube.com/watch?v=Py2nZYmvTKg',
  'https://science.nasa.gov/universe/stars/'],
 'object': 'chat.completion',
 'choices': [{'index': 0,
   'finish_reason': 'stop',
   'message': {'role': 'assistant',
    'content': 'The Milky Way galaxy contains an estimated 100 to 400 billion stars. This range accounts for the difficulty in detecting all stars, especially faint or low-mass ones[1][3][4].'},
   'delta': {'role': 'assistant', 'content': ''}}]}

In [18]:
# Alternative approaches to work with the response:

# 1. Using the .json() method directly from requests (recommended)
response_json = response.json()
print(f"Type of response.json(): {type(response_json)}")

# 2. Working with specific parts of the response
if 'choices' in response_json:
    # Extract the assistant's message
    assistant_message = response_json['choices'][0]['message']['content']
    print(f"\nAssistant's response: {assistant_message}")

    # Extract other useful information
    model = response_json.get('model', 'Unknown')
    completion_tokens = response_json.get(
        'usage', {}).get('completion_tokens', 0)
    prompt_tokens = response_json.get('usage', {}).get('prompt_tokens', 0)

    print(f"\nModel used: {model}")
    print(
        f"Tokens used: {prompt_tokens} (prompt) + {completion_tokens} (completion) = {prompt_tokens + completion_tokens} (total)")

Type of response.json(): <class 'dict'>

Assistant's response: The Milky Way galaxy contains an estimated 100 to 400 billion stars. This range accounts for the difficulty in detecting all stars, especially faint or low-mass ones[1][3][4].

Model used: sonar
Tokens used: 14 (prompt) + 40 (completion) = 54 (total)


In [ ]:
# Reusable function to handle API responses
def process_llm_api_response(response):
    """
    Process a response from an LLM API and return useful components.

    Args:
        response: The requests.Response object from the API call

    Returns:
        dict: A dictionary containing processed response data
    """
    try:
        # Convert to dictionary if it's not already
        if isinstance(response, requests.Response):
            data = response.json()
        elif isinstance(response, str):
            data = json.loads(response)
        elif isinstance(response, dict):
            data = response
        else:
            raise TypeError(f"Unsupported response type: {type(response)}")

        # Extract common fields from various LLM APIs
        result = {
            'success': True,
            'raw_response': data,
            'content': None,
            'model': data.get('model', 'unknown'),
            'usage': data.get('usage', {})
        }

        # Handle response format differences between APIs
        if 'choices' in data and len(data['choices']) > 0:
            # OpenAI, Perplexity, and similar format
            choice = data['choices'][0]
            if 'message' in choice and 'content' in choice['message']:
                result['content'] = choice['message']['content']
            elif 'text' in choice:
                result['content'] = choice['text']

        return result

    except Exception as e:
        return {
            'success': False,
            'error': str(e),
            'raw_response': response.text if isinstance(response, requests.Response) else response
        }

Response content: The Milky Way galaxy contains an estimated 100 to 400 billion stars. This range accounts for the difficulty in detecting all stars, especially faint or low-mass ones[1][3][4].
Model used: sonar
Total tokens: 54


In [35]:
# Example usage
processed_response = process_llm_api_response(response)
if processed_response['success']:
    print(f"Response content: {processed_response['content']}")
    print(f"Model used: {processed_response['model']}")
    if processed_response['usage']:
        print(
            f"Total tokens: {processed_response['usage'].get('total_tokens', 'unknown')}")
else:
    print(f"Error: {processed_response['error']}")

Error: Unsupported response type: <class 'types.SimpleNamespace'>


## Create Chatlas like Class for Perplexity

In [2]:
import os
import json
import requests
from types import SimpleNamespace
from urllib.parse import urlparse


class ChatPerplexityDirect:
    """
    A class for interacting with the Perplexity API directly, similar to Chatlas but with
    citation handling. This implementation extracts and processes citations from responses.
    """

    def __init__(self, api_key=None, model="sonar-pro", system_prompt=""):
        """
        Initialize the Perplexity API client.

        Args:
            api_key (str): Perplexity API key. If None, will try to get from environment.
            model (str): The model to use. Options: "sonar", "sonar-pro", "sonar-small", "sonar-medium", "claude-3.5-sonnet", etc.
            system_prompt (str): System prompt to use for all conversations.
        """
        self.api_key = api_key or os.getenv("PERPLEXITY_API_KEY")
        if not self.api_key:
            raise ValueError(
                "Perplexity API key is required. Set it with api_key parameter or PERPLEXITY_API_KEY env variable.")

        self.model = model
        self.system_prompt = system_prompt
        self.url = "https://api.perplexity.ai/chat/completions"
        self.headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }

    def _extract_citations(self, response_json):
        """
        Extract citations from a Perplexity API response.

        Args:
            response_json (dict): The JSON response from the Perplexity API.

        Returns:
            list: List of citation dictionaries.
        """
        citations = []

        # Check for top-level citations field first (as in your example)
        if 'citations' in response_json and isinstance(response_json['citations'], list):
            for url in response_json['citations']:
                citation = {
                    'url': url,
                    'title': self._extract_domain_from_url(url),
                    'text': ''  # No excerpt available in this format
                }
                citations.append(citation)
            return citations

        # If no top-level citations, check other possible locations
        if 'choices' in response_json and len(response_json['choices']) > 0:
            choice = response_json['choices'][0]

            # Extract tool calls (citations) if they exist
            if 'message' in choice and 'tool_calls' in choice['message']:
                tool_calls = choice['message']['tool_calls']
                for tool_call in tool_calls:
                    if tool_call.get('type') == 'link':
                        function = tool_call.get('function', {})
                        if function and 'arguments' in function:
                            try:
                                args = json.loads(function['arguments'])
                                if 'url' in args:
                                    citation = {
                                        'url': args['url'],
                                        'title': args.get('title', self._extract_domain_from_url(args['url'])),
                                        'text': args.get('text', '')
                                    }
                                    citations.append(citation)
                            except json.JSONDecodeError:
                                pass

            # Look for citations in special 'links' field if available
            if 'message' in choice and 'links' in choice['message']:
                for link in choice['message']['links']:
                    citation = {
                        'url': link.get('url', ''),
                        'title': link.get('title', self._extract_domain_from_url(link.get('url', ''))),
                        'text': link.get('text', '')
                    }
                    citations.append(citation)

        return citations

    def _extract_domain_from_url(self, url):
        """
        Extract a readable domain name from a URL to use as a title
        when no explicit title is available.

        Args:
            url (str): The URL to extract a domain from

        Returns:
            str: A readable domain name or the original URL if parsing fails
        """
        try:
            parsed_url = urlparse(url)
            # Remove www. if present and return the domain
            domain = parsed_url.netloc
            if domain.startswith('www.'):
                domain = domain[4:]
            # If it's YouTube, try to make it more descriptive
            if 'youtube.com' in domain or 'youtu.be' in domain:
                return "YouTube Video"
            # Wikipedia articles can be more descriptive
            if 'wikipedia.org' in domain and '/wiki/' in url:
                topic = url.split('/wiki/')[1].replace('_', ' ')
                return f"Wikipedia: {topic}"
            return domain
        except:
            return url

    def _create_response_object(self, response_json):
        """
        Create a response object similar to what Chatlas would return.

        Args:
            response_json (dict): The JSON response from the Perplexity API.

        Returns:
            SimpleNamespace: A dot-accessible object with the response data.
        """

        # Extract the main response content
        content = ""
        if 'choices' in response_json and len(response_json['choices']) > 0:
            choice = response_json['choices'][0]
            if 'message' in choice and 'content' in choice['message']:
                content = choice['message']['content']

        # Extract citations
        citations = self._extract_citations(response_json)

        # Create a Chatlas-like response object
        response_obj = SimpleNamespace(
            content=content,
            raw_response=response_json,
            model=response_json.get('model', self.model),
            citations=citations,
            usage=response_json.get('usage', {})
        )

        return response_obj

    def chat(self, message, echo=None):
        """
        Send a message to the Perplexity API and get a response.

        Args:
            message (str): The user's message to send to the API.
            echo (str): If "all", print both request and response. If "response", print only response. 
                      If "none" or None, print nothing.

        Returns:
            SimpleNamespace: A response object with content, citations, and metadata.
        """
        # Prepare the messages array
        messages = []
        if self.system_prompt:
            messages.append({"role": "system", "content": self.system_prompt})
        messages.append({"role": "user", "content": message})

        # Build the payload
        payload = {
            "model": self.model,
            "messages": messages,
            # Add parameters to enhance citations
            "tools": [{"type": "link"}],  # Request link citations
            "tool_choice": "auto",  # Let the model decide when to add citations
            "link_history": True    # Include history of links
        }

        # Echo the request if requested
        if echo == "all":
            print("Request:")
            print(json.dumps(payload, indent=2))
            print("\n")

        # Make the API call
        response = requests.post(self.url, json=payload, headers=self.headers)
        response_json = response.json()

        # Echo the response if requested
        if echo in ["all", "response"]:
            print("Response:")
            print(json.dumps(response_json, indent=2))
            print("\n")

        # Process and return the response
        return self._create_response_object(response_json)

### Example usage

In [3]:
perplexity = ChatPerplexityDirect(model="sonar-pro")
response = perplexity.chat(
    "What are the latest treatments for heart failure? Please provide scientific citations.", echo="response")

Response:
{
  "id": "92607008-4438-4d28-b9af-80b6c0d95405",
  "model": "sonar-pro",
  "created": 1747748323,
  "usage": {
    "prompt_tokens": 14,
    "completion_tokens": 554,
    "total_tokens": 568,
    "search_context_size": "low"
  },
  "citations": [
    "https://www.crf.org/crf/news-and-events/news/news/3959-tht-2025-late-breaking-clinical-science-unveiled",
    "https://flowtherapy.com/resource/latest-advances-in-heart-failure-treatment/",
    "https://www.escardio.org/Congresses-Events/Heart-Failure",
    "https://clinicaltrials.ucsd.edu/heart-failure",
    "https://news.northwestern.edu/stories/2025/04/injectable-therapy-could-prevent-heart-failure-after-a-heart-attack/"
  ],
  "object": "chat.completion",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "Several innovative treatments for heart failure have emerged in recent years, offering new hope for patients who haven't responded well 

In [4]:
print(f"\nResponse content:\n{response.content}")
print("-"*80)
print(f"\nModel used: {response.model}")


Response content:
Several innovative treatments for heart failure have emerged in recent years, offering new hope for patients who haven't responded well to traditional therapies.

## Pharmacological Advances

Ivabradine represents an important advancement in heart failure medication. This selective inhibitor of the I(f) channel in the sinoatrial node helps reduce heart rate, which can improve outcomes in heart failure patients[2].

The combination drug Sacubitril/Valsartan has shown significant promise in treating heart failure. This medication combines a neprilysin inhibitor with an angiotensin receptor blocker, providing dual action to improve heart function[2].

## Device-Based Therapies

**HeartMate 3** is an advanced left ventricular assist device that has revolutionized mechanical circulatory support for heart failure patients. This implantable device helps the heart pump blood more effectively, offering both short and long-term support options[2].

**MitraClip therapy** has em

In [5]:
print(type(response))
response

<class 'types.SimpleNamespace'>


namespace(content="Several innovative treatments for heart failure have emerged in recent years, offering new hope for patients who haven't responded well to traditional therapies.\n\n## Pharmacological Advances\n\nIvabradine represents an important advancement in heart failure medication. This selective inhibitor of the I(f) channel in the sinoatrial node helps reduce heart rate, which can improve outcomes in heart failure patients[2].\n\nThe combination drug Sacubitril/Valsartan has shown significant promise in treating heart failure. This medication combines a neprilysin inhibitor with an angiotensin receptor blocker, providing dual action to improve heart function[2].\n\n## Device-Based Therapies\n\n**HeartMate 3** is an advanced left ventricular assist device that has revolutionized mechanical circulatory support for heart failure patients. This implantable device helps the heart pump blood more effectively, offering both short and long-term support options[2].\n\n**MitraClip ther

### Citations and Usage Info

In [6]:
# CITATIONS
if hasattr(response, 'citations') and response.citations:
    print(f"\nCitations ({len(response.citations)}):\n")
    for i, citation in enumerate(response.citations, 1):
        print(f"{i}. {citation['title']}")
        print(f"   URL: {citation['url']}")
        if citation['text']:
            print(f"   Excerpt: {citation['text'][:100]}..." if len(
                citation['text']) > 100 else citation['text'])
        print()
else:
    print("\nNo citations found in the response.")

# USAGE
if hasattr(response, 'usage') and response.usage:
    print(f"\nTokens used: {response.usage.get('prompt_tokens', 0)} (prompt) + {response.usage.get('completion_tokens', 0)} (completion) = {response.usage.get('total_tokens', 0)} (total)")


Citations (5):

1. crf.org
   URL: https://www.crf.org/crf/news-and-events/news/news/3959-tht-2025-late-breaking-clinical-science-unveiled

2. flowtherapy.com
   URL: https://flowtherapy.com/resource/latest-advances-in-heart-failure-treatment/

3. escardio.org
   URL: https://www.escardio.org/Congresses-Events/Heart-Failure

4. clinicaltrials.ucsd.edu
   URL: https://clinicaltrials.ucsd.edu/heart-failure

5. news.northwestern.edu
   URL: https://news.northwestern.edu/stories/2025/04/injectable-therapy-could-prevent-heart-failure-after-a-heart-attack/


Tokens used: 14 (prompt) + 554 (completion) = 568 (total)
